# Introduction: Automated Feature Engineering

In this notebook, we will look at an exciting development in data science: automated feature engineering. A machine learning model can only learn from the data we give it, and making sure that data is relevant to the task is one of the most crucial steps in the machine learning pipeline (this is made clear in the excellent paper ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)). 

However, manual feature engineering is a tedious task and is limited by both human imagination - there are only so many features we can think to create - and by time - creating new features is time-intensive. Ideally, there would be an objective method to create an array of diverse new candidate features that we can then use for a machine learning task. This process is meant to not replace the data scientist, but to make her job easier and allowing her to supplement domain knowledge with an automated workflow.

In this notebook, we will walk through an implementation of using [Featuretools](https://www.featuretools.com/), an open-source Python library for automatically creating features with relational data (where the data is in structured tables). Although there are now many efforts working to enable automated model selection and hyperparameter tuning, there has been a lack of automating work on the feature engineering aspect of the pipeline. This library seeks to close that gap and the general methodology has been proven effective in both [machine learning competitions with the data science machine](https://github.com/HDI-Project/Data-Science-Machine) and [business use cases](https://www.featurelabs.com/blog/predicting-credit-card-fraud/). 


## Dataset

To show the basic idea of featuretools we will use an example dataset consisting of three tables:

* `clients`: information about clients at a credit union
* `loans`: previous loans taken out by the clients
* `payments`: payments made/missed on the previous loans

The general problem of feature engineering is taking disparate data, often distributed across multiple tables, and combining it into a single table that can be used for training a machine learning model. Featuretools has the ability to do this for us, creating many new candidate features with minimal effort. These features are combined into a single table that can then be passed on to our model. 

First, let's load in the data and look at the problem we are working with.

In [5]:
# Run this if featuretools is not already installed
%pip install -U featuretools
%pip install pandas
%pip install numpy

  Using cached dask-2023.3.2-py3-none-any.whl (1.2 MB)
  Using cached scipy-1.10.1-cp39-cp39-win_amd64.whl (42.5 MB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached distributed-2023.3.2-py3-none-any.whl (956 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached numpy-1.24.2-cp39-cp39-win_amd64.whl (14.9 MB)
  Using cached PyYAML-6.0-cp39-cp39-win_amd64.whl (151 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached partd-1.3.0-py3-none-any.whl (18 kB)
  Using cached toolz-0.12.0-py3-none-any.whl (55 kB)
  Using cached fsspec-2023.3.0-py3-none-any.whl (145 kB)
  Using cached zict-2.2.0-py2.py3-none-any.whl (23 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached locket-1.0.0-py2.py3-none-any.whl (4.4 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached tblib-1.7.0-py2.py3-none-any.whl (12 kB)
  Using cached msgpack-1.0.5-cp39-cp39-win_amd64.whl (62 kB)
  Using cached MarkupSafe

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyvista 0.37.0 requires imageio, which is not installed.
You should consider upgrading via the 'c:\Users\white\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\white\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\white\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Read in the data
clients = pd.read_csv('data/clients.csv', parse_dates = ['joined'])
loans = pd.read_csv('data/loans.csv', parse_dates = ['loan_start', 'loan_end'])
payments = pd.read_csv('data/payments.csv', parse_dates = ['payment_date'])

In [8]:
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [9]:
loans.sample(10)

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
92,25707,cash,12623,1,11790,2004-09-15,2007-01-23,4.09
33,49545,other,11434,0,10864,2001-09-22,2004-05-26,3.28
423,49624,other,13015,1,10233,2002-07-25,2005-03-26,0.32
361,26695,cash,3660,0,10341,2011-12-19,2013-11-21,2.55
88,25707,cash,13913,0,11211,2011-11-24,2014-04-14,0.57
30,49545,home,13618,1,10691,2001-11-17,2003-05-19,2.63
18,46109,credit,12009,0,11424,2001-03-25,2003-10-04,0.79
398,41472,home,5095,0,11406,2010-04-16,2012-08-22,5.00
387,46958,other,9069,0,11226,2004-03-03,2006-04-12,5.80
211,26326,other,6119,0,11513,2005-11-15,2007-06-06,2.35


In [10]:
payments.sample(10)

,loan_id,payment_amount,payment_date,missed
894,11338,442,2009-06-13,0
3409,10812,2530,2010-07-07,1
1336,10933,977,2003-03-23,0
1133,11587,999,2005-04-19,1
195,11866,1586,2002-05-07,0
2164,10240,2022,2007-12-07,1
825,11975,556,2009-11-09,0
1361,11680,2423,2009-12-09,1
25,11415,1813,2010-12-24,0
1925,10424,117,2010-01-09,0


### Manual Feature Engineering Examples

Let's show a few examples of features we might make by hand. We will keep this relatively simple to avoid doing too much work! First we will focus on a single dataframe before combining them together. In the `clients` dataframe, we can take the month of the `joined` column and the natural log of the `income` column. Later, we see these are known in featuretools as transformation feature primitives because they act on column in a single table. 

In [11]:
# Create a month column
clients['join_month'] = clients['joined'].dt.month

# Create a log of income column
clients['log_income'] = np.log(clients['income'])

clients.head()

,client_id,joined,income,credit_score,join_month,log_income
0,46109,2002-04-16,172677,527,4,12.059178
1,49545,2007-11-14,104564,770,11,11.557555
2,41480,2013-03-11,122607,585,3,11.716739
3,46180,2001-11-06,43851,562,11,10.688553
4,25707,2006-10-06,211422,621,10,12.261611


To incorporate information about the other tables, we use the `df.groupby` method, followed by a suitable aggregation function, followed by `df.merge`.  For example, let's calculate the average, minimum, and maximum amount of previous loans for each client. In the terms of featuretools, this would be considered an aggregation feature primitive because we using multiple tables in a one-to-many relationship to calculate aggregation figures (don't worry, this will be explained shortly!).

In [14]:
# Groupby client id and calculate mean, max, min previous loan size
stats = loans.groupby('client_id')['loan_amount'].agg(['sum'])
stats.columns = ['total_loan_amount']
stats.head()

,min,max,total_loan_amount
client_id,,,
25707,1212,13913,159279
26326,1164,13464,116321
26695,2389,14865,140845
26945,653,14593,106889
29841,2778,14837,176634


In [66]:
# Merge with the clients dataframe
clients.merge(stats, left_on = 'client_id', right_index=True, how = 'left').head(10)

,client_id,joined,income,credit_score,join_month,log_income,total_loan_amount
0,46109,2002-04-16,172677,527,4,12.059178,179032
1,49545,2007-11-14,104564,770,11,11.557555,205786
2,41480,2013-03-11,122607,585,3,11.716739,157897
3,46180,2001-11-06,43851,562,11,10.688553,154017
4,25707,2006-10-06,211422,621,10,12.261611,159279
5,39505,2011-10-14,153873,610,10,11.943883,148481
6,32726,2006-05-01,235705,730,5,12.370336,126032
7,35089,2010-03-01,131176,771,3,11.784295,138784
8,35214,2003-08-08,95849,696,8,11.470529,129124
9,48177,2008-06-09,190632,769,6,12.158100,141063


We could go further and include information about `payments` in the `clients` dataframe. To do so, we would have to group `payments` by the `loan_id`, merge it with the `loans`, group the resulting dataframe by the `client_id`, and then merge it into the `clients` dataframe. This would allow us to include information about previous payments for each client. 

Clearly, this process of manual feature engineering can grow quite tedious with many columns and multiple tables and I certainly don't want to have to do this process by hand! Luckily, featuretools can automatically perform this entire process and will create more features than we would have ever thought of. Although I love `pandas`, there is only so much manual data manipulation I'm willing to stand! 

# Featuretools

Now that we know what we are trying to avoid (tedious manual feature engineering), let's figure out how to automate this process. Featuretools operates on an idea known as [Deep Feature Synthesis](https://docs.featuretools.com/api_reference.html#deep-feature-synthesis). You can read the [original paper here](http://www.jmaxkanter.com/static/papers/DSAA_DSM_2015.pdf), and although it's quite readable, it's not necessary to understand the details to do automated feature engineering. The concept of Deep Feature Synthesis is to use basic building blocks known as feature primitives (like the transformations and aggregations done above) that can be stacked on top of each other to form new features. The depth of a "deep feature" is equal to the number of stacked primitives. 

I threw out some terms there, but don't worry because we'll cover them as we go. Featuretools builds on simple ideas to create a powerful method, and we will build up our understanding in much the same way. 

The first part of Featuretools to understand [is an `entity`](https://docs.featuretools.com/loading_data/using_entitysets.html#adding-entities). This is simply a table, or in `pandas`, a `DataFrame`. We corral multiple entities into a [single object called an `EntitySet`](https://docs.featuretools.com/loading_data/using_entitysets.html). This is just a large data structure composed of many individual entities and the relationships between them.  

## EntitySet

Creating a new `EntitySet` is pretty simple: 

In [67]:
es = ft.EntitySet(id = 'clients')

## Entities 

An entity is simply a table, which is represented in Pandas as a `dataframe`. Each entity must have a uniquely identifying column, known as an index. For the clients dataframe, this is the `client_id` because each id only appears once in the `clients` data. In the `loans` dataframe, `client_id` is not an index because each id might appear more than once. The index for this dataframe is instead `loan_id`. 

When we create an `entity` in featuretools, we have to identify which column of the dataframe is the index. If the data does not have a unique index we can tell featuretools to make an index for the entity by passing in `make_index = True` and specifying a name for the index. If the data also has a uniquely identifying time index, we can pass that in as the `time_index` parameter. 

Featuretools will automatically infer the variable types (numeric, categorical, datetime) of the columns in our data, but we can also pass in specific datatypes to override this behavior. As an example, even though the `repaid` column in the `loans` dataframe is represented as an integer, we can tell featuretools that this is a categorical feature since it can only take on two discrete values. This is done using an integer with the variables as keys and the feature types as values.

In the code below we create the three entities and add them to the `EntitySet`.  The syntax is relatively straightforward with a few notes: for the `payments` dataframe we need to make an index, for the `loans` dataframe, we specify that `repaid` is a categorical variable, and for the `payments` dataframe, we specify that `missed` is a categorical feature. 

In [68]:
# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.add_dataframe(dataframe_name = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [69]:
from woodwork.logical_types import Categorical
# Create an entity from the loans dataframe
# This dataframe already has an index and a time index
es = es.add_dataframe(dataframe_name = 'loans', dataframe = loans, 
                              logical_types = {'repaid': Categorical},
                              index = 'loan_id', 
                              time_index = 'loan_start')

In [70]:
from woodwork.logical_types import Categorical
# Create an entity from the payments dataframe
# This does not yet have a unique index
es = es.add_dataframe(dataframe_name = 'payments', 
                              dataframe = payments,
                              logical_types = {'missed': Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

In [71]:
es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    No relationships

All three entities have been successfully added to the `EntitySet`. We can access any of the entities using Python dictionary syntax.

In [72]:
es['clients']

,client_id,joined,income,credit_score,join_month,log_income
42320,42320,2000-04-27,229481,563,4,12.343576
39384,39384,2000-06-18,191204,617,6,12.161096
26945,26945,2000-11-26,214516,806,11,12.276140
41472,41472,2001-11-06,152214,638,11,11.933043
46180,46180,2001-11-06,43851,562,11,10.688553
46109,46109,2002-04-16,172677,527,4,12.059178
32885,32885,2002-05-13,58955,642,5,10.984530
29841,29841,2002-08-17,38354,523,8,10.554614
38537,38537,2002-10-21,127183,643,10,11.753382
35214,35214,2003-08-08,95849,696,8,11.470529


In [73]:
es['loans']

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
11140,39505,home,2274,1,11140,2000-01-26,2002-01-29,1.00
11251,26326,home,2847,1,11251,2000-03-06,2001-09-26,1.32
10816,49545,home,8354,1,10816,2000-03-08,2001-08-02,0.45
11965,29841,credit,6012,0,11965,2000-03-25,2002-07-10,4.63
10166,41472,home,13657,1,10166,2000-04-11,2001-09-08,5.68
...,...,...,...,...,...,...,...,...
11595,35089,other,773,1,11595,2014-09-26,2016-04-23,7.63
10985,26695,other,13918,1,10985,2014-10-03,2016-10-25,0.90
10684,48177,credit,659,1,10684,2014-10-05,2017-01-16,1.52
10131,49068,other,10082,1,10131,2014-10-10,2016-05-25,0.63


Featuretools correctly inferred each of the datatypes when we made this entity. We can also see that we overrode the type for the `repaid` feature, changing if from numeric to categorical. 

In [74]:
es['payments']

,payment_id,loan_id,payment_amount,payment_date,missed
2113,2113,11988,2053,2000-03-05,0
726,726,11140,402,2000-03-19,0
2114,2114,11988,2627,2000-03-30,0
3223,3223,11430,1284,2000-04-05,0
2115,2115,11988,1911,2000-04-11,1
...,...,...,...,...,...
1415,1415,11072,957,2015-07-01,0
1308,1308,10684,115,2015-07-06,0
1416,1416,11072,988,2015-07-14,1
1417,1417,11072,940,2015-07-29,0


## Relationships

After defining the entities (tables) in an `EntitySet`, we now need to tell featuretools [how they are related with a relationship](https://docs.featuretools.com/loading_data/using_entitysets.html#adding-a-relationship). The most intuitive way to think of relationships is with the parent to child analogy: a parent-to-child relationship is one-to-many because for each parent, there can be multiple children. The `client` dataframe is therefore the parent of the `loans` dataframe because while there is only one row for each client in the `client` dataframe, each client may have several previous loans covering multiple rows in the `loans` dataframe. Likewise, the `loans` dataframe is the parent of the `payments` dataframe because each loan will have multiple payments. 

These relationships are what allow us to group together datapoints using aggregation primitives and then create new features. As an example, we can group all of the previous loans associated with one client and find the average loan amount. We will discuss the features themselves more in a little bit, but for now let's define the relationships. 

To define relationships, we need to specify the parent variable and the child variable. This is the variable that links two entities together. In our example, the `client` and `loans` dataframes are linked together by the `client_id` column. Again, this is a parent to child relationship because for each `client_id` in the parent `client` dataframe, there may be multiple entries of the same `client_id` in the child `loans` dataframe. 

We codify relationships in the language of featuretools by specifying the parent variable and then the child variable. After creating a relationship, we add it to the `EntitySet`. 

In [77]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(entityset=es,parent_dataframe_name='clients',parent_column_name='client_id',
                    child_dataframe_name='loans',child_column_name='client_id')

# Add the relationship to the entity set
es = es.add_relationship(relationship=r_client_previous)

The relationship has now been stored in the entity set. The second relationship is between the `loans` and `payments`. These two entities are related by the `loan_id` variable.

In [76]:
# Relationship between previous loans and previous payments
r_payments = ft.Relationship(entityset=es,parent_dataframe_name='loans',parent_column_name='loan_id',
                    child_dataframe_name='payments',child_column_name='loan_id')

# Add the relationship to the entity set
es = es.add_relationship(relationship=r_payments)

es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

We now have our entities in an entityset along with the relationships between them. We can now start to making new features from all of the tables using stacks of feature primitives to form deep features. First, let's cover feature primitives.


## Feature Primitives

A [feature primitive](https://docs.featuretools.com/automated_feature_engineering/primitives.html) a at a very high-level is an operation applied to data to create a feature. These represent very simple calculations that can be stacked on top of each other to create complex features. Feature primitives fall into two categories:

* __Aggregation__: function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation. An example is calculating the maximum loan amount for each client. An aggregation works across multiple tables using relationships between tables.
* __Transformation__: an operation applied to one or more columns in a single table. An example would be extracting the day from dates, or finding the difference between two columns in one table.

Let's take a look at feature primitives in featuretools. We can view the list of primitives:

In [78]:
# checking relationship
es

Entityset: clients
  DataFrames:
    clients [Rows: 25, Columns: 6]
    loans [Rows: 443, Columns: 8]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

In [79]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
0,max_consecutive_positives,aggregation,False,False,Determines the maximum number of consecutive positive values in the input,"<ColumnSchema (Logical Type = Integer)>, <ColumnSchema (Logical Type = Double)>",<ColumnSchema (Logical Type = Integer) (Semantic Tags = ['numeric'])>
1,average_count_per_unique,aggregation,False,False,Determines the average count across all unique value.,<ColumnSchema (Semantic Tags = ['category'])>,<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
2,all,aggregation,True,False,Calculates if all values are 'True' in a list.,"<ColumnSchema (Logical Type = Boolean)>, <ColumnSchema (Logical Type = BooleanNullable)>",<ColumnSchema (Logical Type = Boolean)>
3,time_since_last_true,aggregation,False,False,Calculates the time since the last `True` value.,"<ColumnSchema (Logical Type = BooleanNullable)>, <ColumnSchema (Logical Type = Boolean)>, <Colum...",<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
4,sum,aggregation,True,True,"Calculates the total addition, ignoring `NaN`.",<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
5,max,aggregation,True,True,"Calculates the highest value, ignoring `NaN` values.",<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
6,correlation,aggregation,False,False,Computes the correlation between two columns of values.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
7,count,aggregation,True,True,"Determines the total number of values, excluding `NaN`.",<ColumnSchema (Semantic Tags = ['index'])>,<ColumnSchema (Logical Type = IntegerNullable) (Semantic Tags = ['numeric'])>
8,count_above_mean,aggregation,False,False,Calculates the number of values that are above the mean.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = IntegerNullable) (Semantic Tags = ['numeric'])>
9,num_unique,aggregation,True,True,"Determines the number of distinct values, ignoring `NaN` values.",<ColumnSchema (Semantic Tags = ['category'])>,<ColumnSchema (Logical Type = IntegerNullable) (Semantic Tags = ['numeric'])>


In [80]:
primitives[primitives['type'] == 'transform'].head(10)

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
54,rolling_max,transform,False,False,Determines the maximum of entries over a given window.,"<ColumnSchema (Logical Type = Datetime) (Semantic Tags = ['time_index'])>, <ColumnSchema (Semant...",<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
55,longitude,transform,False,False,Returns the second tuple value in a list of LatLong tuples. For use with the LatLong logical type.,<ColumnSchema (Logical Type = LatLong)>,<ColumnSchema (Semantic Tags = ['numeric'])>
56,minute,transform,True,True,Determines the minutes value of a datetime.,<ColumnSchema (Logical Type = Datetime)>,"<ColumnSchema (Logical Type = Ordinal: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16..."
57,natural_logarithm,transform,True,True,Computes the natural logarithm of a number.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
58,greater_than_equal_to_scalar,transform,True,True,Determines if values are greater than or equal to a given scalar.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = BooleanNullable)>
59,lag,transform,False,False,Shifts an array of values by a specified number of periods.,"<ColumnSchema (Logical Type = Boolean)>, <ColumnSchema (Semantic Tags = ['numeric'])>, <ColumnSc...",None
60,number_of_unique_words,transform,False,False,Determines the number of unique words in a string.,<ColumnSchema (Logical Type = NaturalLanguage)>,<ColumnSchema (Logical Type = IntegerNullable) (Semantic Tags = ['numeric'])>
61,exponential_weighted_variance,transform,False,False,Computes the exponentially weighted moving variance for a series of numbers,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>
62,part_of_day,transform,True,True,Determines the part of day of a datetime.,<ColumnSchema (Logical Type = Datetime)>,<ColumnSchema (Logical Type = Categorical) (Semantic Tags = ['category'])>
63,cosine,transform,True,True,Computes the cosine of a number.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = Double) (Semantic Tags = ['numeric'])>


If featuretools does not have enough primitives for us, we can [also make our own.](https://docs.featuretools.com/automated_feature_engineering/primitives.html#defining-custom-primitives) 

To get an idea of what a feature primitive actually does, let's try out a few on our data. Using primitives is surprisingly easy using the `ft.dfs` function (which stands for deep feature synthesis). In this function, we specify the entityset to use; the `target_entity`, which is the dataframe we want to make the features for (where the features end up); the `agg_primitives` which are the aggregation feature primitives; and the `trans_primitives` which are the transformation primitives to apply. 

In the following example, we are using the `EntitySet` we already created, the target entity is the `clients` dataframe because we want to make new features about each client, and then we specify a few aggregation and transformation primitives. 

In [90]:
# Create new features using specified primitives
features, feature_names = ft.dfs(entityset = es, target_dataframe_name = 'clients', 
                                 agg_primitives = ['mean', 'max', 'percent_true', 'last', 'sum'],
                                 trans_primitives = ["part_of_day"])

In [95]:
pd.DataFrame(features['PART_OF_DAY(LAST(payments.payment_date))'].head())

,PART_OF_DAY(LAST(payments.payment_date))
client_id,
42320,midnight
39384,midnight
26945,midnight
41472,midnight
46180,midnight


In [84]:
pd.DataFrame(features['MEAN(payments.payment_amount)'].head())

,MEAN(payments.payment_amount)
client_id,
42320,1021.483333
39384,1193.630137
26945,1109.473214
41472,1129.076190
46180,1186.550336


In [85]:
features.head()

,income,credit_score,join_month,log_income,LAST(loans.loan_amount),LAST(loans.loan_id),LAST(loans.loan_type),LAST(loans.rate),LAST(loans.repaid),MAX(loans.loan_amount),...,LAST(payments.loans.repaid),MAX(payments.loans.loan_amount),MAX(payments.loans.rate),MEAN(payments.loans.loan_amount),MEAN(payments.loans.rate),SUM(payments.loans.loan_amount),SUM(payments.loans.rate),PART_OF_DAY(LAST(loans.loan_end)),PART_OF_DAY(LAST(loans.loan_start)),PART_OF_DAY(LAST(payments.payment_date))
client_id,,,,,,,,,,,,,,,,,,,,,
42320,229481,563,4,12.343576,8090,10156,home,3.18,0,13887.0,...,0,13887.0,6.74,7028.058333,2.523667,843367.0,302.84,midnight,midnight,midnight
39384,191204,617,6,12.161096,14654,11735,other,2.26,0,14654.0,...,0,14654.0,9.23,7957.130137,3.418630,1161741.0,499.12,midnight,midnight,midnight
26945,214516,806,11,12.276140,9249,11482,cash,2.86,1,14593.0,...,1,14593.0,5.65,6884.401786,2.947589,771053.0,330.13,midnight,midnight,midnight
41472,152214,638,11,11.933043,10122,11936,cash,1.03,0,13657.0,...,0,13657.0,9.82,7473.628571,4.146286,784731.0,435.36,midnight,midnight,midnight
46180,43851,562,11,10.688553,3834,10887,other,1.38,0,14081.0,...,0,14081.0,9.26,7668.899329,3.882081,1142666.0,578.43,midnight,midnight,midnight


Already we can see how useful featuretools is: it performed the same operations we did manually but also many more in addition. Examining the names of the features in the dataframe brings us to the final piece of the puzzle: deep features.

## Deep Feature Synthesis

While feature primitives are useful by themselves, the main benefit of using featuretools arises when we stack primitives to get deep features. The depth of a feature is simply the number of primitives required to make a feature. So, a feature that relies on a single aggregation would be a deep feature with a depth of 1, a feature that stacks two primitives would have a depth of 2 and so on. The idea itself is lot simpler than the name "deep feature synthesis" implies. (I think the authors were trying to ride the way of deep neural network hype when they named the method!) To read more about deep feature synthesis, check out [the documentation](https://docs.featuretools.com/automated_feature_engineering/afe.html) or the [original paper by Max Kanter et al](http://www.jmaxkanter.com/static/papers/DSAA_DSM_2015.pdf). 

Already in the dataframe we made by specifying the primitives manually we can see the idea of feature depth. For instance, the MEAN(loans.loan_amount) feature has a depth of 1 because it is made by applying a single aggregation primitive. This feature represents the average size of a client's previous loans.

In [48]:
# Show a feature with a depth of 1
pd.DataFrame(features['MEAN(loans.loan_amount)'].head(10))

,MEAN(loans.loan_amount)
client_id,
42320,7062.066667
39384,7865.473684
26945,7125.933333
41472,7510.812500
46180,7700.850000
46109,8951.600000
32885,9920.400000
29841,9813.000000
38537,8986.352941


As well scroll through the features, we see a number of features with a depth of 2. For example, the LAST(loans.(MEAN(payments.payment_amount))) has depth = 2 because it is made by stacking two feature primitives, first an aggregation and then a transformation. This feature represents the average payment amount for the last (most recent) loan for each client.

In [49]:
# Show a feature with a depth of 2
pd.DataFrame(features['LAST(loans.MEAN(payments.payment_amount))'].head(10))

,LAST(loans.MEAN(payments.payment_amount))
client_id,
42320,1192.333333
39384,2311.285714
26945,1598.666667
41472,1427.000000
46180,557.125000
46109,1708.875000
32885,1729.000000
29841,1125.500000
38537,1348.833333


We can create features of arbitrary depth by stacking more primitives. However, when I have used featuretools I've never gone beyond a depth of 2! After this point, the features become very convoluted to understand. I'd encourage anyone interested to experiment with increasing the depth (maybe for a real problem) and see if there is value to "going deeper".

## Automated Deep Feature Synthesis

In addition to manually specifying aggregation and transformation feature primitives, we can let featuretools automatically generate many new features. We do this by making the same `ft.dfs` function call, but without passing in any primitives. We just set the `max_depth` parameter and featuretools will automatically try many all combinations of feature primitives to the ordered depth. 

When running on large datasets, this process can take quite a while, but for our example data, it will be relatively quick. For this call, we only need to specify the `entityset`, the `target_entity` (which will again be `clients`), and the `max_depth`. 

In [51]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_dataframe_name='clients', 
                                 max_depth = 2)

In [52]:
features.iloc[:, 4:].head()

,COUNT(loans),MAX(loans.loan_amount),MAX(loans.rate),MEAN(loans.loan_amount),MEAN(loans.rate),MIN(loans.loan_amount),MIN(loans.rate),MODE(loans.loan_type),MODE(loans.repaid),NUM_UNIQUE(loans.loan_type),...,MODE(payments.loans.loan_type),MODE(payments.loans.repaid),NUM_UNIQUE(payments.loans.loan_type),NUM_UNIQUE(payments.loans.repaid),SKEW(payments.loans.loan_amount),SKEW(payments.loans.rate),STD(payments.loans.loan_amount),STD(payments.loans.rate),SUM(payments.loans.loan_amount),SUM(payments.loans.rate)
client_id,,,,,,,,,,,,,,,,,,,,,
42320,15,13887.0,6.74,7062.066667,2.457333,1070.0,0.38,home,1,4,...,home,1,4,2,0.097213,0.872916,3929.351652,1.891948,843367.0,302.84
39384,19,14654.0,9.23,7865.473684,3.538421,1770.0,0.43,credit,1,4,...,cash,1,4,2,-0.277360,0.971757,3765.368569,2.685701,1161741.0,499.12
26945,15,14593.0,5.65,7125.933333,2.855333,653.0,0.13,credit,0,4,...,credit,0,4,2,0.289824,0.020986,4372.558827,1.599556,771053.0,330.13
41472,16,13657.0,9.82,7510.812500,3.981250,986.0,0.01,cash,0,4,...,other,1,4,2,-0.064213,0.311596,4250.127784,3.129716,784731.0,435.36
46180,20,14081.0,9.26,7700.850000,3.502500,1607.0,0.57,other,0,4,...,other,0,4,2,0.127445,0.649145,3815.702946,2.714351,1142666.0,578.43


Deep feature synthesis has created 90 new features out of the existing data! While we could have created all of these manually, I am glad to not have to write all that code by hand. The primary benefit of featuretools is that it creates features without any subjective human biases. Even a human with considerable domain knowledge will be limited by their imagination when making new features (not to mention time). Automated feature engineering is not limited by these factors (instead it's limited by computation time) and provides a good starting point for feature creation. This process likely will not remove the human contribution to feature engineering completely because a human can still use domain knowledge and machine learning expertise to select the most important features or build new features from those suggested by automated deep feature synthesis.

# Next Steps

While automatic feature engineering solves one problem, it provides us with another problem: too many features! Although it's difficult to say which features will be important to a given machine learning task ahead of time, it's likely that not all of the features made by featuretools add value. In fact, having too many features is a significant issue in machine learning because it makes training a model much harder. The [irrelevant features can drown out the important features](https://pdfs.semanticscholar.org/a83b/ddb34618cc68f1014ca12eef7f537825d104.pdf), leaving a model unable to learn how to map the features to the target.

This problem is known as the ["curse of dimensionality"](https://en.wikipedia.org/wiki/Curse_of_dimensionality#Machine_learning) and is addressed through the process of [feature reduction and selection](http://scikit-learn.org/stable/modules/feature_selection.html), which means [removing low-value features](https://machinelearningmastery.com/feature-selection-machine-learning-python/) from the data. Defining which features are useful is an important problem where a data scientist can still add considerable value to the feature engineering task. Feature reduction will have to be another topic for another day!

# Conclusions

In this notebook, we saw how to apply automated feature engineering to an example dataset. This is a powerful method which allows us to overcome the human limits of time and imagination to create many new features from multiple tables of data. Featuretools is built on the idea of deep feature synthesis, which means stacking multiple simple feature primitives - __aggregations and transformations__ - to create new features. Feature engineering allows us to combine information across many tables into a single dataframe that we can then use for machine learning model training. Finally, the next step after creating all of these features is figuring out which ones are important. 

Featuretools is currently the only Python option for this process, but with the recent emphasis on automating aspects of the machine learning pipeline, other competitiors will probably enter the sphere. While the exact tools will change, the idea of automatically creating new features out of existing data will grow in importance. Staying up-to-date on methods such as automated feature engineering is crucial in the rapidly changing field of data science. Now go out there and find a problem on which to apply featuretools! 

For more information, check out the [documentation for featuretools](https://docs.featuretools.com/index.html). Also, read about how featuretools is [used in the real world by Feature Labs](https://www.featurelabs.com/), the company behind the open-source library.